# L4 – Word embedding

### Resources
1. Possible [truncated svd](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) implementation of sklearn.
2. Tensorflow [tutorial](https://www.tensorflow.org/tutorials/word2vec) for word2vec and [implementation](https://github.com/tensorflow/models/tree/master/tutorials/embedding).
3. GloVe [site](https://nlp.stanford.edu/projects/glove/).
4. FastText [site](https://fasttext.cc).
5. [Gensim](https://radimrehurek.com/gensim/) library.

**Word embedding** – the collective name for a set of language modeling and feature learning techniques in natural language processing where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with much higher dimensionality. Word and phrase embeddings, when used as the underlying input representation, have been shown to boost the performance in NLP tasks such as syntactic parsing, sentiment analysis and translation. Usually, problem reduces to unsupervised learning on a large corpus of text. So, learning process is based on the idea that context of word closely associated with it. For example, the context can be a document in which the word is located or adjacent words.

### 1. Data

As text corpus you may use simple english [wikipedia](https://simple.wikipedia.org/wiki/Simple_English_Wikipedia).
1. 131K articles
2. 2K common English words.

Also you can use just english [wikipedia](https://en.wikipedia.org/wiki/Wikipedia):
1. Includes 5М+ articles
2. 2M+ different tokens

#### Exercises
1. Download all wikipedia articles.
2. Make all preprocessing work that you need (e.g. remove markup).

In [16]:
import tensorflow as tf
import gensim
import pattern
import sklearn.decomposition
import scipy.sparse
from scipy.spatial.distance import cdist
import numpy as np
import collections
from tqdm import tqdm
import random
import pickle

from gensim.corpora import WikiCorpus, MmCorpus, Dictionary
from gensim import corpora, models, similarities

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

We will filter out words which are found is less than 7 articles and those which are attributed to more than 40% of all articles in corpus.

In [43]:
DUMP_NAME = "simplewiki-20180201-pages-articles.xml.bz2"
wiki_corpus = WikiCorpus(DUMP_NAME, lemmatize=False)
wiki_corpus.dictionary.filter_extremes(no_below=7, no_above=0.4, keep_n=100000)
wiki_corpus.dictionary.save('filtered_wiki.dict')

2018-02-09 16:46:56,858 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-02-09 16:47:04,003 : INFO : adding document #10000 to Dictionary(145816 unique tokens: ['bankruptcy', 'donlin', 'suebians', 'transtromer', 'ot']...)
2018-02-09 16:47:10,233 : INFO : adding document #20000 to Dictionary(207404 unique tokens: ['bankruptcy', 'donlin', 'suebians', 'woodsboro', 'praiseland']...)
2018-02-09 16:47:17,919 : INFO : adding document #30000 to Dictionary(264020 unique tokens: ['bankruptcy', 'donlin', 'suebians', 'collamer', 'woodsboro']...)
2018-02-09 16:47:26,104 : INFO : adding document #40000 to Dictionary(304936 unique tokens: ['bankruptcy', 'donlin', 'suebians', 'collamer', 'woodsboro']...)
2018-02-09 16:47:32,796 : INFO : adding document #50000 to Dictionary(358223 unique tokens: ['bankruptcy', 'donlin', 'nothosaur', 'collamer', 'boucherie']...)
2018-02-09 16:47:39,689 : INFO : adding document #60000 to Dictionary(411366 unique tokens: ['bankruptcy', 'donlin', 'nothos

In [47]:
wiki_corpus.save('simple_wiki_corpus')

/usr/local/lib/python3.5/dist-packages/gensim/interfaces.py:60: UserWarning: corpus.save() stores only the (tiny) iteration object; to serialize the actual corpus content, use e.g. MmCorpus.serialize(corpus)
  "corpus.save() stores only the (tiny) iteration object; "
2018-02-09 16:56:14,530 : INFO : saving WikiCorpus object under simple_wiki_corpus, separately None
2018-02-09 16:56:14,572 : INFO : saved simple_wiki_corpus


In [3]:
wiki_corpus = WikiCorpus.load('simple_wiki_corpus')

2018-02-10 08:56:21,384 : INFO : loading WikiCorpus object from simple_wiki_corpus
2018-02-10 08:56:21,534 : INFO : loading dictionary recursively from simple_wiki_corpus.dictionary.* with mmap=None
2018-02-10 08:56:21,537 : INFO : loaded simple_wiki_corpus


### 2. LSA (Latent semantic analysis)
This solution uses full document as a context of word. So, we have some vocabulary $W$ and a set of documents $D$. Matrix $X$ with shape $|W| \times |D|$ at position $w, d$ stores importance of word $w$ for document $d$. If word $w$ is not found in the document $d$ than at appropriate position $X$ has 0 (obviously, matrix is sparse).

For each matrix you can find [SVD decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)
$$X = U \Sigma V^{T} \text{, где }$$
* $U$ – orthogonal matrix $|W| \times |W|$ of left singular vectors
* $\Sigma$ – diagonal matrix $|W| \times |D|$ of singular values
* $V$ – orthogonal matrix $|D| \times |D|$  of right singular vectors

Let's suppouse that row $w$ in matrix $U\Sigma$ is a vector that represents word $w$, and row $d$ of $V$ coresponds to document $d$. In some sense we already found the embeddings of words and documents at the same time. But size of vectors are determined by documents number $|D|$.

Nevertheless you can use truncated SVD instead
$$ X \approx X_k = U_k \Sigma_k V^{T}_k \text{, where }$$
* $U_k$ – $k$ left singular vectors
* $\Sigma_k$ – diagonal matrix $|k| \times |k|$ of largest singular values
* $V_k$ – $k$ right singular vectors

Also it known that $X_k$ is the best approximation of $X$ in the term of [Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm) for all matrices of rank $k$.

So, it is possible to obtain a compressed words' representations of size $k \ll |W|$ as rows of matrix $U_k \Sigma_k $. It just remains to understand how to calculate the original values of the matrix $X$. There is a set of approaches, here are some of them
1. Binary flag that document contains the word.
2. The number occurrences of word in document.
3. Word frequency that is better known as $tf$ (it's possible also use $\ln(1 + tf)$).
4. [$tf \cdot idf$](https://ru.wikipedia.org/wiki/TF-IDF).

#### Further readning
1. You also can read this [aticle](https://en.wikipedia.org/wiki/Latent_semantic_analysis).
2. By the way, some [modification](http://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) of SVD decompostion won at [Netflix prize](https://en.wikipedia.org/wiki/Netflix_Prize).
3. It is easy to see that the problem of SVD decomposition is reduced to finding eigenvectors of matrix $X \cdot X^T$.
4. You can find [here](https://arxiv.org/abs/0909.4061) probabilistic algorithms of matrix decomposition.

#### Exercises
1. Find matrix $X$, using you favourite approach (maybe you need the sparse matrix class of scipy library).
2. Find word embeddings of size $k = 128$.
3. Implement k-nearest neighbor search for Euclidean norm.
4. Find 10 nearest words for the set of words: 'cat', 'loves', 'clever' and 'mandarin'.
5. Repeat experiment for cosinus as metric. What are the results?

Let's firstly find $X$ matrix by using tf-idf approach:

In [ ]:
tfidf = models.TfidfModel(wiki_corpus, id2word=wiki_corpus.dictionary)
wiki_tfidf = tfidf[wiki_corpus]
wiki_sparse = gensim.matutils.corpus2csc(wiki_tfidf)

As we have sparse matrix, we can find word embeddings by reducing dimensionality to $R^{128}$

In [110]:
DIM = 128

In [74]:
svd = sklearn.decomposition.TruncatedSVD(n_components=DIM, n_iter=7, random_state=1773)
embeddings = svd.fit_transform(wiki_sparse)

Let's save embeddings and sparse matrix in order to use them when reloading kernel.

In [81]:
np.save('word_embeddings.npy', embeddings)

In [83]:
with open('wiki_sparse_tfidf.pickle', 'wb') as f:
    pickle.dump(wiki_sparse, f)

We're now ready to implement kNN and play with words:

In [142]:
def get_k_nearest(word, embeddings, dictionary, k, metric):
    def id2word(idx):
        return dictionary[idx]
    
    def word2id(word):
        return dictionary.doc2idx([word])[0]
    
    distances = cdist(embeddings, 
                      embeddings[word2id(word)].reshape(1, embeddings.shape[1]),
                      metric=metric).flatten()
        
    return [(id2word(x), distances[x]) for x in np.argsort(distances)[:k + 1] if x != word2id(word)][:k]

In [143]:
def print_k_nearest(word, embeddings, dictionary, k, metric):
    nearest_list = get_k_nearest(word, embeddings, dictionary, k, metric)
    print('Nearest to', word)
    print('%15s %10s' %('word', 'distance'))
    print('='*30)
    for word_info in nearest_list:
        print("%15s | %.4f" %(word_info[0], word_info[1]))
    print('='*30, end='\n\n')

In [201]:
print_k_nearest('cat', embeddings, wiki_corpus.dictionary, 10, 'euclidean')
print_k_nearest('loves', embeddings, wiki_corpus.dictionary, 10, 'euclidean')
print_k_nearest('clever', embeddings, wiki_corpus.dictionary, 10, 'euclidean')
print_k_nearest('mandarin', embeddings, wiki_corpus.dictionary, 10, 'euclidean')

Nearest to cat
           word   distance
           cats | 0.3558
            dog | 0.4080
          breed | 0.4174
           bear | 0.4245
         spider | 0.4298
           dogs | 0.4322
           eyes | 0.4328
            fur | 0.4356
           duck | 0.4549
     appearance | 0.4558

Nearest to loves
           word   distance
          funny | 0.1129
          likes | 0.1129
           ugly | 0.1254
            mom | 0.1260
      wonderful | 0.1260
           sees | 0.1275
           wish | 0.1280
          knows | 0.1289
          proud | 0.1297
          curse | 0.1308

Nearest to clever
           word   distance
        realize | 0.0395
        jealous | 0.0405
         honest | 0.0435
       persuade | 0.0435
     frightened | 0.0436
          wakes | 0.0441
     dostoevsky | 0.0442
        shocked | 0.0444
          cared | 0.0445
         liking | 0.0446

Nearest to mandarin
           word   distance
     simplified | 0.1181
      cantonese | 0.1386
      taiwanese | 0

Let's change metric to cosine and compare results:

In [195]:
print_k_nearest('cat', embeddings, wiki_corpus.dictionary, 10, 'cosine')
print_k_nearest('loves', embeddings, wiki_corpus.dictionary, 10, 'cosine')
print_k_nearest('clever', embeddings, wiki_corpus.dictionary, 10, 'cosine')
print_k_nearest('mandarin', embeddings, wiki_corpus.dictionary, 10, 'cosine')

Nearest to cat
           word   distance
           cats | 0.1244
            dog | 0.1496
            pet | 0.1554
        kittens | 0.1598
           paws | 0.1783
          tabby | 0.1957
           ears | 0.1993
         rabbit | 0.2100
       whiskers | 0.2102
        siamese | 0.2176

Nearest to loves
           word   distance
         friend | 0.1268
        friends | 0.1354
          happy | 0.1744
           ugly | 0.1892
          funny | 0.1900
          likes | 0.1904
           girl | 0.2024
            mom | 0.2061
       realises | 0.2061
          proud | 0.2093

Nearest to clever
           word   distance
         really | 0.1686
         thinks | 0.1713
        telling | 0.1714
          knows | 0.1721
          angry | 0.1810
        jealous | 0.1847
        realize | 0.1995
          proud | 0.2014
       dislikes | 0.2068
            sad | 0.2099

Nearest to mandarin
           word   distance
      cantonese | 0.0334
     simplified | 0.0863
        hokkien | 0

As we can see, cosine metric performs a little bit better meaning that nearest words are generally more relevant, comparing to results with euclidean metric.

### 3. word2vec
Let's consider perhaps the most popular model of word embeddings. This is mainly due to the fact that the received vectors have interesting properties. In particular, semantically similar words have close vectors, moreover linear operation of subtraction has meaning in the resulting vector space! The context of word at this time is a window of size $2c+1$, where interested word is located in the middle.

#### Continuous bag-of-words
The core idea is very simple. There is some very long text
$$w_1, w_2, \dots, w_T.$$
Consider some word $w_t$ and its context in radius $c$, namely words
$$w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}.$$
Intuition tells us that the context often really well defines the word in the middle. Then let our model restore the central word if its context is known
$$\mathcal{L} = - \frac{1}{T}\sum_{t} \log \mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}].$$

For each word $w_t$ there is some vector $v_t$, but if word $w_{t+i}$ is some context than we use vector $v'_{t+i}$. In this case we can represent context of word as the following sum
$$s'_t = \sum_{c \leq i \leq c, i \neq 0} v'_{t+i}$$

And define probabilty with softmax
$$\mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t-1}, \dots w_{t+c}] = \frac{ \exp {s'}_t^T \cdot v_t}{\sum_j \exp {s'}_t^T \cdot v_j}.$$

Note that this model is easy to present as 2-layer neural network:
* The input is a sparse vector of dimension $|W|$ which at position $w'$ has one if word $w'$ is in context.
* Further, we have matrix $V'$ of vectors $v'$.
* Further, without any non-linearity matrix $V$ of vector $v$.
* And then the softmax layer.

#### Skip-gram
Another model predicts context for some word. The design is approximately the same, we need to optimize
$$\mathcal{L} = -\frac{1}{T} \sum_t \sum_{-c \leq i \leq c, i \neq 0} \mathbb{P}[w_{t+i}|w_t].$$

The probability is approximated by the following model
$$\mathbb{P}[w_{t+i}|w_t] = \frac{ \exp {v'}_{t+i}^T \cdot v_t}{\sum_j \exp {v'}_{j}^T \cdot v_t}.$$


#### Hierarchical softmax
Creation of this mode was guided to reduce the complexity of learning. However, computing of sofmax's denominator is really expensive operation. Therefore, in practice, a few other models are used.

This method was viewed in the seminar. Detailed information can be found [here](http://www.iro.umontreal.ca/~lisa/pointeurs/hierarchical-nnlm-aistats05.pdf) and practical using [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).    Briefly recall, for each word from the dictionary we define [Huffman code](https://en.wikipedia.org/wiki/Huffman_coding). More frequency words have shorter code. Then we construct the Huffman tree. In accordance with its code words are stored in leaves. In the non-leaf vertices are stored special hidden representations $h'$. 

$$\mathbb{P}[w_{t+i}|w_t] = \prod_{{h'}_{t+i}, r} \sigma\Big( \mathcal{r} \cdot {h'}^T_{t+i} v_t \Big) \text{, where}$$
* ${h'}_{t+i}$ - hidden vector of word's path $w_{t+i}$
* $\mathcal{r}$ - equals $-1$, if we turn left and $+1$ in another case
* $\sigma$ - sigmoid function

Using hierarchical softmax as approximation of softmax allows us to significantly reduce complexity of model training. 

#### Exercise
1. How can you implement skip-gram model as neural network.
2. Estimate complexity of one iteration of training the skip-gram model
  * $T$ - text size
  * $W$ - vocabulary size
  * $c$ - context radius
  * $d$ - embedding size
3. Estimate complexity of using hierarchical softmax in the skip-gram model.

#### Why sampling?
Let see at softmax funciton
$$\mathbb{P}[x_i] = \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)}$$
Optimizing this function is equivalent to optimizing its logarithm
$$\log \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)} = f(x_i|\theta) - log \sum_j \exp f(x_j|\theta)$$
Than gradient $\triangledown_{\theta} \mathbb{P}[x_i]$ is equal to
$$\triangledown_{\theta} f(x_i|\theta) - \sum_k \frac{\exp f(x_k|\theta)}{\sum_j \exp f(w_j|\theta)} \triangledown_{\theta} f(x_k|\theta)$$
Inside the sum we can find $\mathbb{P}[x_k]$ again. So, the previous expression can be written as
$$
\triangledown_{\theta} f(x_i|\theta) - \sum_k \mathbb{P}[x_k] \cdot \triangledown_{\theta} f(x_k|\theta) = 
\triangledown_{\theta} f(x_i|\theta) - \mathbb{E}_{x \sim \mathbb{P}[x]} \triangledown_{\theta} f(x|\theta)
$$
If we can sample $x \sim \mathbb{P}[x]$, then we has no sense to calculate the softmax itself. We just can train model, computing only gradient. It is possible due to the fact that we do not need the probability itself, but want to find representation of words only.

#### Negative Sampling
As the result, in practice, softmax loss functions is approximated with negative sampling, which looks like a series of logistic regressions
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t}
    \sum_{-c \leq i \leq c, i \neq 0} \Big(
        \mathbb{P}[y = 1 | w_{t+i}, w_t] -
        k \cdot \mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]} \mathbb{P}[y = 0 | \tilde{w}_{tj}, w_t]
    \Big)
$$
where $y=1$, if  word is a context of word and $y=0$ in another case. А $\tilde{w}_{tj}$ – sampling of words that don't belong to the context. Then main trick is how we will sample words $\mathbb{P}[y | w_{t+i}, w_t]$. In paper [noise contrastive estimation](https://www.cs.helsinki.fi/u/ahyvarin/papers/Gutmann10AISTATS.pdf) some interesting approaches is suggested. But we are going to use more simple variation
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t} \Big(
    \sum_{-c \leq i \leq c, i \neq 0} \big(
        \sigma({v'}_{t+i}^T v_t) +
        k \cdot \mathbb{E}_{w_j \sim \mathbb{Q}(w_j)} \sigma (-{v'}_{j}^T v_t)
    \big)
\Big),
$$
to find  $\mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]}$ we simply sample words $k$ times from distribution $\mathbb{Q}$, $\mathbb{P}(w_j|w_t) = \mathbb{Q}(w_j)$.

#### Additional word2vec tricks
For the best quality at the moment of learning it is proposed to use variety of tricks:
1. How to choose $\mathbb{Q}$.
2. Sampling of frequent and rare words.
3. The sampling of window size.
You are welcome to read about this in original papers.

#### Further reading
1. Briefly about both models [here](https://arxiv.org/pdf/1301.3781.pdf).
2. Hierarchical softmax and negative sampling [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

#### Exercises
1. Train skip-gram model, using negative sampling and embedding size $d$ = 256. You are free to choose size of window and number of epochs.
2. Find top-10 nearest words again. Compare Euclidean and cosinus metric. Which is better? 
3. Find 10 nearest vector for expression v(king) - v(man) + v(women). If your model is tuned well, you find representation of word queen. 
4. How could you solve the following problems?
  * Find for country its capital
  * For a word find its plural form
  * For word define antonyms

In [10]:
filtered_voc = Dictionary.load('filtered_wiki.dict')
vocabulary_set = set(filtered_voc.values())
vocabulary_size = len(vocabulary_set)

2018-02-10 08:59:07,949 : INFO : loading Dictionary object from filtered_wiki.dict
2018-02-10 08:59:08,111 : INFO : loaded filtered_wiki.dict


In [11]:
def build_dataset(corpus, vocabulary):
    words = []
    
    print('INFO: extracting words from corpus')
    for text in tqdm(corpus.get_texts()):
        words.extend([word for word in text if word in vocabulary])
    count = []
    count.extend(collections.Counter(words).most_common())
    dictionary = dict()
    
    print('INFO: creating dictionary')
    for word, _ in tqdm(count):
        dictionary[word] = len(dictionary)
        
    data = list()
    print('INFO: creating dataset')
    for word in tqdm(words):
        index = dictionary[word]
        data.append(index)
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [12]:
t_data, t_count, t_dict, t_rev_dict = build_dataset(wiki_corpus, vocabulary_set)

0it [00:00, ?it/s]

INFO: extracting words from corpus


87304it [01:35, 918.50it/s] 2018-02-10 09:00:53,529 : INFO : finished iterating over Wikipedia corpus of 87384 documents with 23340546 positions (total 243051 articles, 24725459 positions before pruning articles shorter than 50 words)
87384it [01:35, 917.68it/s]
  0%|          | 0/16506837 [00:00<?, ?it/s]

INFO: creating dictionary
INFO: creating dataset


100%|██████████| 16506837/16506837 [00:13<00:00, 1256376.18it/s]


In [17]:
with open('t_data.pickle', 'wb') as f:
    pickle.dump(t_data, f)
with open('t_dict.pickle', 'wb') as f:
    pickle.dump(t_dict, f)
with open('t_rev_dict.pickle', 'wb') as f:
    pickle.dump(t_rev_dict, f)   

In [ ]:
with open('t_data.pickle', 'rb') as f:
    t_data = pickle.load(f)
with open('t_dict.pickle', 'rb') as f:
    t_dict = pickle.load(f)
with open('t_rev_dict.pickle', 'rb') as f:
    t_rev_dict = pickle.load(f)

In [62]:
data_index = 0

def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
            
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

Size of the whole corpus in words:

In [63]:
len(t_data)

16506837

Initializing our graph:

In [66]:
batch_size = 512
embedding_size = 256
skip_window = 4
num_skips = 8
num_sampled = 64

w2vec_graph = tf.Graph()

with w2vec_graph.as_default():
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        
    with tf.name_scope('embeddings'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    with tf.name_scope('weights'):
        nce_weights = tf.Variable(tf.truncated_normal(
            [vocabulary_size, embedding_size],
            stddev=1.0 / np.sqrt(embedding_size)))
        
    with tf.name_scope('biases'):
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocabulary_size))
        
    tf.summary.scalar('loss', loss)
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
    
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    merged = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [69]:
EPOCHS = 5
num_steps = int((len(t_data) * num_skips * EPOCHS) / batch_size)

In [71]:
with tf.Session(graph=w2vec_graph) as session:
    init.run()
    print('Successfully initialized')

    average_loss = 0
    for step in tqdm(range(num_steps)):
        batch_inputs, batch_labels = generate_batch(t_data, batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss],
                              feed_dict=feed_dict)
        average_loss += loss_val

        if step % 100000 == 0:
            if step > 0:
                average_loss /= 100000
            # The average loss is an estimate of the loss over the last 100000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0
    final_embeddings_norm = normalized_embeddings.eval()
    final_embeddings = embeddings.eval()


  0%|          | 0/1289596 [00:00<?, ?it/s]

Successfully initialized


  0%|          | 22/1289596 [00:00<12:29:34, 28.67it/s]

Average loss at step  0 :  282.5571594238281


  8%|▊         | 100030/1289596 [08:14<1:38:05, 202.13it/s]

...

 85%|████████▌ | 1100037/1289596 [1:34:27<16:16, 194.10it/s]

Average loss at step  1100000 :  4.37682833045423


 93%|█████████▎| 1200032/1289596 [1:42:57<07:41, 194.27it/s]

Average loss at step  1200000 :  4.360913152610138


100%|██████████| 1289596/1289596 [1:50:45<00:00, 194.05it/s]


Saving models in order to use it after restarting kernel:

In [72]:
np.save('w2v_norm_embed', final_embeddings_norm)
np.save('w2v_embed', final_embeddings)

In [21]:
final_embeddings_norm = np.load('w2v_norm_embed.npy')
final_embeddings = np.load('w2v_embed.npy')

In [87]:
class SkipGramModel:
    def __init__(self, embeddings, word2id_dict, id2word_dict, normalised_embeddings=None):
        self.embeddings = embeddings
        self.normalised_embeddings = normalised_embeddings
        self.word2id_dict = word2id_dict
        self.id2word_dict = id2word_dict
    
    def print_nearest(self, word, neighb_n, metric, use_normalised=False):
        embeddings = self.embeddings
        if use_normalised and self.normalised_embeddings is not None:
            embeddings = self.normalised_embeddings
        word2id_dict = self.word2id_dict
        id2word_dict = self.id2word_dict
        distances = cdist(embeddings,
                          embeddings[word2id_dict[word]].reshape(1, embeddings.shape[1]),
                          metric=metric).flatten()

        nearest_list = [(id2word_dict[x], distances[x]) for x in 
                        np.argsort(distances)[:neighb_n + 1] if x != word2id_dict[word]][:neighb_n]

        print('Nearest to', word)
        print('%15s %10s' %('word', 'distance'))
        print('='*30)
        for word_info in nearest_list:
            print("%15s | %.4f" %(word_info[0], word_info[1]))
        print('='*30, end='\n\n')
    
    def print_sub_add_nearest(self, in_word, sub_word, add_word, neighb_n, metric, use_normalised=False):
        embeddings = self.embeddings
        if use_normalised and self.normalised_embeddings is not None:
            embeddings = self.normalised_embeddings
        word2id_dict = self.word2id_dict
        id2word_dict = self.id2word_dict
        input_word_ids = [word2id_dict[x] for x in [in_word, sub_word, add_word]]
        new_word_embed = embeddings[word2id_dict[in_word]] - embeddings[word2id_dict[sub_word]] + embeddings[word2id_dict[add_word]]
        distances = cdist(embeddings, 
                          new_word_embed.reshape(1, embeddings.shape[1]),
                          metric=metric).flatten()
        nearest_list = [(id2word_dict[x], distances[x]) for x in 
                        np.argsort(distances)[:neighb_n + 3] if x not in input_word_ids][:neighb_n]

        print('Nearest to %s - %s + %s' % (in_word, sub_word, add_word))
        print('%15s %10s' %('word', 'distance'))
        print('='*30)
        for word_info in nearest_list:
            print("%15s | %.4f" %(word_info[0], word_info[1]))
        print('='*30, end='\n\n')

In [89]:
sgm = SkipGramModel(final_embeddings, t_dict, t_rev_dict, final_embeddings_norm)
sgm.print_nearest('cat', 10, 'euclidean')
sgm.print_nearest('loves', 10, 'euclidean')
sgm.print_nearest('clever', 10, 'euclidean')
sgm.print_nearest('mandarin', 10, 'euclidean')

Nearest to cat
           word   distance
           cats | 4.2294
            dog | 4.3203
           face | 4.6912
          breed | 4.7318
        friends | 4.7472
            man | 4.7570
       probably | 4.7633
            now | 4.7661
       actually | 4.7740
          white | 4.7745

Nearest to loves
           word   distance
           love | 4.5964
          wants | 4.6275
           sees | 4.7487
         thinks | 4.7531
            him | 4.7593
          tells | 4.8218
       actually | 4.8347
      everybody | 4.8473
           them | 4.8668
          loved | 4.8695

Nearest to clever
           word   distance
         really | 5.7128
           good | 5.7470
          happy | 5.8393
           said | 5.8568
            fun | 5.8741
        however | 5.8998
            but | 5.9081
            say | 5.9280
         simple | 5.9456
        because | 5.9582

Nearest to mandarin
           word   distance
        chinese | 5.1909
        english | 5.4828
           hong | 5

In [91]:
sgm.print_nearest('cat', 10, 'cosine')
sgm.print_nearest('loves', 10, 'cosine')
sgm.print_nearest('clever', 10, 'cosine')
sgm.print_nearest('mandarin', 10, 'cosine')

Nearest to cat
           word   distance
           cats | 0.3666
            dog | 0.4341
          breed | 0.4411
       domestic | 0.5476
            fur | 0.5532
        friends | 0.5652
        cartoon | 0.5668
          white | 0.5706
            pet | 0.5755
           girl | 0.5758

Nearest to loves
           word   distance
          wants | 0.4460
           sees | 0.4635
         thinks | 0.4721
      everybody | 0.4826
          tells | 0.4933
       believes | 0.4935
          tries | 0.4943
          likes | 0.4970
        decides | 0.4972
           love | 0.4984

Nearest to clever
           word   distance
    intelligent | 0.5373
         really | 0.5481
           good | 0.5740
           knew | 0.5809
         stupid | 0.5821
            fun | 0.5827
          happy | 0.5827
     interested | 0.5922
          proud | 0.5986
          think | 0.6018

Nearest to mandarin
           word   distance
        chinese | 0.4362
      cantonese | 0.5053
           hong | 0

It's clear, that `cosine` metric gives us a bit better results semantically, for instance, `intelligent` is intuitively much closer to `clever` than `really` or `good`.

In [94]:
sgm.print_sub_add_nearest('king', 'man', 'woman', 10, 'cosine')

Nearest to king - man + woman
           word   distance
          queen | 0.4088
            she | 0.5234
       princess | 0.5237
            her | 0.5253
      elizabeth | 0.5307
         female | 0.5314
        kingdom | 0.5404
          women | 0.5654
       margaret | 0.5664
           lady | 0.5674



Let's solve some funny tasks:

Firstly, we will find the capital for a country:

In [95]:
sgm.print_sub_add_nearest('athens', 'greece', 'norway', 10, 'cosine')

Nearest to athens - greece + norway
           word   distance
           oslo | 0.4196
          munch | 0.5373
        private | 0.6150
         danish | 0.6187
         berlin | 0.6195
      norwegian | 0.6203
     cincinnati | 0.6212
   philadelphia | 0.6226
            men | 0.6243
         museum | 0.6250



Finding plural form of a word:

In [111]:
sgm.print_sub_add_nearest('cars', 'car', 'woman', 10, 'cosine')

Nearest to cars - car + woman
           word   distance
          women | 0.4522
            men | 0.5229
            man | 0.5376
         female | 0.5504
           ones | 0.5763
        however | 0.5774
           them | 0.5778
         wonder | 0.5785
            she | 0.5846
           they | 0.5852



Finding antonyms of a word:

In [127]:
sgm.print_sub_add_nearest('little', 'big', 'high', 10, 'cosine')

Nearest to little - big + high
           word   distance
            low | 0.4920
         school | 0.5021
         higher | 0.5030
         junior | 0.5523
         normal | 0.5537
          girls | 0.5588
           boys | 0.5736
        grammar | 0.5824
          level | 0.5841
          young | 0.5853



### 4. GloVe
This model combines two ideas:
1. Factorization of the matrix, as in LSA
2. Context of word is adjacent words in the corpus, just in word2vec.

Let matrix $X$ at position $i,j$ stores how many times the word $j$ was found in the context of word $i$. This matrix is filled with our corpus of text. Then the probability that the word $j$ appears in the context of word $i$ is equal to
$$P_{ij} = \frac{X_{ij}}{\sum_k X_{ik}}$$

At the same time we want to find some function $F$ and word embeddings that 
$$F((w_i - w_j)^T w_k) = \frac{F(w_i^T w_k)}{F(w_j^T w_k)} = \frac{P_{ik}}{P_{jk}}.$$
About motivation it's better to read in the original [paper](http://nlp.stanford.edu/pubs/glove.pdf).
Possible solution may be following
$$\exp(w_i^T w_j) = P_{ij}.$$

After a series of transformations you can receive that
$$w_i^T w_j = \log X_{ij} - \log\big(\sum_k X_{ik}\big)$$

Obviously, right part doesn't depend on $j$, but the resulting model is offered to be written symmetry as
$$w_i^T w_j + b_i + b_j = \log X_{ij}$$

As loss function authors suggest to use weighted MSE
$$\mathcal{L} = \sum_{i,j = 1}^{|W|} f(X_{ij}) \Big(w_i^T w_j + b_i + b_j - \log(X_{ij}) \Big)^2,$$
where $f$ is some weight function, defined for each pair of words. It is reasonable to store $X$ as sparse matrix. It is also noteworthy that during training only **not null** values of $X$ used. Model is trained with stochastic gradient descent. As some way of regularization the authors suggest to use two different matrices $W$ и $W'$ for word and context, so model can be rewritten as
$$w_i^T {w'}_j + b_i + b_j = \log X_{ij}$$
The resulted embedding of a word is the sum of two trained views $w + w'$.

#### Some additional GloVe tricks
1. In order to understand how you can choose $f$, please, read the original paper.
2. Also you are welcome to see, how $X$ is formed in the original article.

#### Further reading
1. GloVe [site](http://nlp.stanford.edu/projects/glove/).
2. Some words about [t-SNE](http://lvdmaaten.github.io/tsne/).

#### Exercises
1. Estimate complexity of model for one iteration. Choose appropriate $c$ for you.
2. Train GloVe word embeddings ($d$=256).
3. Check that v(king) - v(man) + v(women) is approximately equal v(queen).
4. Read about [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding).
5. Use t-SNE to reduce the size of embeddings to 3. Make sure that the following groups of vectors are collinear (use visualization)
  * [man, woman], [Mr., Ms], [king, queen], etc
  * [CEO, company]
  * [adjective, its comparative form]

### 5. FastText
[FastText](https://arxiv.org/pdf/1607.04606.pdf) is a logical development of skip-gram model. The core feature is to present the word as the sum of its n-grams embeddings and its own embedding vector. For example, if we want to use only 3-grams, than word **fast** be reprented as n-grams **-fa**, **fas**, **ast**, **st-** and word **-fast-**. The context is encoeded in usual way, as a result similarity of word and context word is the following sum
$$s(w, c) = \sum_{n \in N_w} z^{T}_n v'_{c},$$
where $N_w$ is the set of all n-grams of word $w$ combined with word itself. The authors argue that the use of combination of 3-,4- and 5-grams helps to get better embeddings for rare and even unknown words (emdedding of word itself is null vector). For model training is proposed to use negative sampling.

You can find more information on [site](https://fasttext.cc) of FastText.

#### Exercises
1. Train FastText word embeddings ($d$=256).
2. Find some rare words in your corpus and find top-5 nearest words for them, using cosinus as metric.
3. Compare results with classic word2vec.
4. Invent some funny new word and found its top-5 nearest words again.
5. How could you compare all models. Suggest your metric (see papers for inspiration).